In [16]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [17]:
# get data
data = pd.read_csv('Level_3a_NonPath_SP1_ZyMo_PSL_1_2_179.csv') 

# split data
X=data.iloc[:,1:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

         Xc1.A      Xc1.R  ...  Grantham.Xd.29  Grantham.Xd.30
0    14.767060  10.829177  ...        0.032505        0.032749
1    11.728905   8.138424  ...        0.031626        0.032946
2     8.319776   2.773259  ...        0.031596        0.031098
3    10.255686   3.778411  ...        0.031188        0.031233
4     6.745884   2.698354  ...        0.031939        0.030060
..         ...        ...  ...             ...             ...
174   1.942887   3.157192  ...        0.029667        0.028973
175   3.204075   3.450542  ...        0.030044        0.026204
176  13.834749   3.132396  ...        0.031815        0.029722
177   4.081263   3.361040  ...        0.032362        0.030237
178   6.898783   3.695777  ...        0.032141        0.032526

[179 rows x 150 columns]
0      1
1      1
2      1
3      1
4      1
      ..
174    2
175    2
176    2
177    2
178    2
Name: Class, Length: 179, dtype: category
Categories (2, int64): [1, 2]


In [18]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [19]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

#sm = SMOTE(random_state=45)
#x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X_ld, yy, test_size=0.2,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(150, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


In [20]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(2 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 1, 150)            91200     
_________________________________________________________________
lstm_7 (LSTM)                (None, 1, 128)            142848    
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 283,586
Trainable params: 283,586
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
5/5 [==============================] - 14s 687ms/step - loss: 0.6886 - accuracy: 0.9041 - val_loss: 0.6736 - val_accuracy: 1.0000
Epoch 2/1000
5/5 [==============================] - 0s 20ms/step - loss: 0.6714 - accuracy: 0.9284 - val_loss: 0.6464 - val_accuracy: 0.9722
Epoch 3/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.6422 - accuracy: 0.9752 - val_loss: 0.6034 - val_accuracy: 0.9722
Epoch 4/1000
5/5 [==============================] - 0s 17ms/step - loss: 0.6002 - accuracy: 0.9830 - val_loss: 0.5416 - val_accuracy: 0.9722
Epoch 5/1000
5/5 [==============================] - 0s 19ms/step - loss: 0.5483 - accuracy: 0.9711 - val_loss: 0.4678 - val_accuracy: 0.9722
Epoch 6/1000
5/5 [==============================] - 0s 17ms/step - loss: 0.4712 - accuracy: 0.9528 - val_loss: 0.3916 - val_accuracy: 0.9722
Epoch 7/1000
5/5 [==============================] - 0s 17ms/step - loss: 0.4291 - accuracy: 0.9277 - val_loss: 0.3138 - val_accuracy: 0.9722
Epoch 8/100

In [23]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 1.0
test-acc = 0.9722222222222222
train-CFR =               precision    recall  f1-score   support

           0       1.00      1.00      1.00        22
           1       1.00      1.00      1.00       121

    accuracy                           1.00       143
   macro avg       1.00      1.00      1.00       143
weighted avg       1.00      1.00      1.00       143

test-CFR =               precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      0.97      0.98        30

    accuracy                           0.97        36
   macro avg       0.93      0.98      0.95        36
weighted avg       0.98      0.97      0.97        36

